In [1]:
%cd /content/drive/MyDrive/Research/dataset/hf_dataset

/content/drive/MyDrive/Research/dataset/hf_dataset


In [ ]:
!ls -l --block-size=G

total 68G
-rw------- 1 root root  1G Dec 18 15:01 '0. download_data.ipynb'
-rw------- 1 root root  1G Dec 21 02:48 '1. train_tokenizer.ipynb'
-rw------- 1 root root 14G Dec 18 12:23  clean_date_categories.csv
-rw------- 1 root root  3G Dec 18 12:11  clean_nepberta_data.zip
-rw------- 1 root root 26G Dec 18 14:40  IRIISNEPAL_Nepali_Text_Corpus.csv
-rw------- 1 root root 26G Dec 20 16:38  iriis_text.txt
drwx------ 2 root root  1G Dec 20 16:31  NepaliBPE
-rw------- 1 root root  0G Dec 21 02:34  nepberta_text.txt


## Data cleaning script

In [ ]:
# Probably should `clean_data(new_crawled_data)` before merging `new_crawled_data.csv` and previous`cleaned_data.csv`

from bs4 import BeautifulSoup
import csv
import json
import re
import string
import sys
import pandas as pd

csv.field_size_limit(sys.maxsize)

class CleanData:
    def __init__(self):
        pass
        # return text
    # Example of removing HTML tags
    def clean_html(self, text):
        '''
        # HTML Tag Removal:
        * removes html tags like: <h1>
        * Removes css or js code inside <style> and <script> tags
        '''
        soup = BeautifulSoup(text, "lxml")

        # Remove all <script> and <style> tags
        for script_or_style in soup(['script', 'style']):
            script_or_style.decompose()

        # Get text from the modified HTML
        text = soup.get_text(separator=' ', strip=True)
        # print(text)

        return text

    def convert_to_devanagari_digits(self, input_string):
        # Function to convert 0-9 to ० - ९
        # i.e. Mapping of ASCII digits to Devanagari digits
        devanagari_digits = {
            '0': '०',
            '1': '१',
            '2': '२',
            '3': '३',
            '4': '४',
            '5': '५',
            '6': '६',
            '7': '७',
            '8': '८',
            '9': '९'
        }
        # Convert each digit in the input string
        result = ''.join(devanagari_digits[char] if char in devanagari_digits else char for char in input_string)
        return result

    def remove_non_devanagari_characters(self, text, keep_special_characters=True):
        '''
            # Function to find nepali sequences.
            * keep punctuations if they occur between devanagari characters.
            * Remove punctuation if previous character is not devanagari.
            # Examples
            texts = [
                "उनले दुहेको दूध बेच्नका लागि बजार असाध्यै सानो थियो त्यसैले उनले चीज बनाउने विचार गरे। \"hi there\". what is your name? उनले दुहेको दूध",
                "\"hi there. \"उनले दुहेको\" दूध बेच्नका लागि बजार असाध्यै सानो थियो त्यसैले उनले चीज बनाउने विचार गरे। hi there. what is your name? उनले दुहेको दूध\"",
                "name? उनले दुहेको दूध\""    #output: (last quatation, name?) should be ignored
                ]

            for text in texts:
                removed = remove_non_devanagari_characters(text)
                print(f'text: {text}, \nclen: {removed}\n\n')


            # output
            text: उनले दुहेको दूध बेच्नका लागि बजार असाध्यै सानो थियो त्यसैले उनले चीज बनाउने विचार गरे। "hi there". what is your name? उनले दुहेको दूध,
            clen: उनले दुहेको दूध बेच्नका लागि बजार असाध्यै सानो थियो त्यसैले उनले चीज बनाउने विचार गरे।             उनले दुहेको दूध


            text: "hi there. "उनले दुहेको" दूध बेच्नका लागि बजार असाध्यै सानो थियो त्यसैले उनले चीज बनाउने विचार गरे। hi there. what is your name? उनले दुहेको दूध",
            clen:    "उनले दुहेको दूध बेच्नका लागि बजार असाध्यै सानो थियो त्यसैले उनले चीज बनाउने विचार गरे।             उनले दुहेको दूध"


            text: name? उनले दुहेको दूध",
            clen:  उनले दुहेको दूध"
        '''
        def is_devanagari(char):
            pattern=r'[ऀ-ॿ]'
            return bool(re.match(pattern, char))

        if not keep_special_characters:
            return re.sub(r"[^ऀ-ॿ ]", " ", text)

        sequences = []
        sequence = ''
        punctuation_symbols = string.punctuation    # '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        prefix_punctuations = '\"\'(<[{'
        index=0
        while index < len(text):
            char = text[index]
            if is_devanagari(char) or char == ' ':
                # Character is devanagari
                sequence += char
            elif char in punctuation_symbols:
                # Character is punctuation
                if sequence != '':
                    if (len(text) > index+1) and not is_devanagari(text[index+1]):
                        # e.g. गरे। "hi there" : skip quotation before hi
                        pass
                    else:
                        sequence += char    # Sequence is no empty. i.e. previous char/sequence was devanagari otherwise ignore  punctuation
                elif (len(text) > index+1) and is_devanagari(text[index+1]):
                    # preserve prefix punctuations in devanagari. e.g. """there. \"उनले "": preserve double-quotation before उनले
                    sequence = char + text[index+1]
                    index += 1  # another 1 is added at the end
            else:
                if sequence:
                    sequences.append(sequence)
                    sequence = ''   # Reset sequence
            index += 1

            # print(f'{sequences}\n{sequence}\n{char}{is_devanagari(char)}\n\n')
        if sequence:    # last sequence
            sequences.append(sequence)
        return ' '.join(sequences)
        # Example of using regex for special character removal

    def normalize_data(self, text):
      '''
        * Standerize special characters
        * e.g. convert different types of quotes to standard quotes
      '''
      characters_to_replace = {
        '₹': 'रु',
        'ʻ': "'",
        'ː': ':',
        '？': '?',
        '‟': '"',
        '“' : '"',
        '”': '"',
        '`': "'",
        '৷': '।',
        'ˈ': "'",
        '՛': "'",
        'ǃ': '!',
        '（': '(',
        '：': ':',
        'ˍ': '_',
        '﹣': '-',
        '״': '"',
        'ꞌ': "'",
        '₋': '-',
        '％': '%',
        '꞉': ':',
        '‵': "'"
      }
      # Replace each character in the dictionary with its corresponding standard character
      for char, replacement in characters_to_replace.items():
          text = text.replace(char, replacement)

      return text

    def clean_data(self, text):
        # Remove HTML tags
        text = self.clean_html(text)

        # Normalize some characters
        text = self.normalize_data(text)

        # Convert convert 0-9 to ० - ९
        text = self.convert_to_devanagari_digits(text)

        text = self.remove_non_devanagari_characters(text, keep_special_characters=True)
        # text = text.lower() # No lower characters in devanagari

        # Replace one or more spaces with a single space
        text = re.sub(r'\s+', ' ', text).strip()

        return text

# Clean data and save to 'cleaned_data'
data_cleaner = CleanData()

# if __name__=="__main__":
#     # Assuming `data` is your raw dataset
#     data0 = ["<html><h1>Hi there</h1><style>color:red</style><script>console.log('hello world');</script>Example text</html>", "<div>Another example with special chars: !@#</div>"]
#     data = ['<html><h1>सुर्खेत र    </h1><style> जुम्लामा बाहेक </style><script>console.log("कर्णालीका अरू जिल्लामा");</script>      कर्णालीका अरू जिल्लामा शिशुका लागि आवश्यक एनआईसीयू सेवा नै उपलब्ध छैन।', 'नेपालले करिब एक महिना अघि नै औपचारिक पत्र पठाएर जीबी राईलाई स्वदेश फर्काइदिन गरेको आग्रहबारे मलेशियाले कुनै औपचारिक जबाफ दिएको छैन।', '2024 बीबीसी। अन्य वेबसाइटका सामग्रीहरूका लागि बीबीसी जिम्मेवार छैन।', 'संसदीय छानबिन समिति गठन नभएसम्म संसद्\u200cको कारबाही अघि बढ्न नदिने प्रतिपक्षी दलको अडानका कारण यसअघिको संसद् अधिवेशनको अन्त्यतिरका कामकारबाही प्रभावित भएका थिए।', 'भारतमा पनि भेपमा प्रतिबन्ध लगाइएको छ तर स्थानीय सञ्चार माध्यमका अनुसार यो व्यवस्था प्रभावकारी रूपमा कार्यान्वयन भएको छैन।', 'इजरेलले रफाहमा आक्रमण गर्न सक्ने आशङ्काका बीच अमेरिकाले गत साता इजरेललाई उपलब्ध गराउन लागेको हजारौँ बमको ढुवानी रोकिदिएको खुलासा भएको छ।', 'शुक्रवारको प्रयत्न सफल भएमा अन्तर्राष्ट्रिय अन्तरिक्ष स्टेशनतिर र बाट उडान गर्ने बोइङ दोस्रो निजी कम्पनी बन्नेछ। अहिलेसम्म इलोन मस्कको स्पेस एक्सले त्यस्तो काम गर्दै आएको छ।', 'हिरादेवी खतिवडाले दैनिक परामर्श दिनेहरूमा जबरजस्ती करणी, यौन दुर्व्यवहार लगायतका घटनाका पीडितहरू हुने गर्छन्।', 'युक्रेनी सेक्युरिटी सर्भिसका अनुसार गिरफ्तार गरिएका दुईजना कर्णेलले जेलेन्स्कीका अङ्गरक्षकहरूमध्ये त्यस्ता व्यक्तिको खोजी गरिरहेका थिए जो उनको अपहरण तथा हत्या गर्नका निम्ति इच्छुक होऊन्।', 'आधिकारिक रूपमा पुटिनले मार्चमा भएको राष्ट्रपतीय चुनावमा ८७ प्रतिशतभन्दा धेरै मत हासिल गरेका थिए। उक्त चुनावमा उनले गम्भीर प्रतिद्वन्दीको सामना गर्नुपरेको थिएन, न उक्त चुनावलाई धेरैले स्वतन्त्र र निष्पक्ष चुनावका रूपमा हेरेका थिए।', 'नेपालका ग्रामीण भेगमा विगतमा पाइने गरेका काठका पुराना ठेकीहरू अचेल नपाइने गरेका मानिसहरू बताउँछन्। आखिर कहाँ जादै छन् सारा ठेकीहरू?', 'चीनको अर्थतन्त्र सोचेभन्दा तीव्र गतिमा विकसित भए पनि यसका अगाडि विभिन्न सङ्कट रहेकाले \nमहिलाहरू घरखर्च धान्न बचत बढाउन चाहन्छन्।', "चार वर्षअघि जारी नेपालको पछिल्लो नक्सालाई अन्तर्राष्ट्रिय मान्यता दिलाउन उल्लेख्य प्रगति नभएको अवस्थामा सरकारको यो निर्णय 'लोकप्रियताका लागि मात्रै गरिएको' कतिपय विज्ञहरूको टिप्पणी छ।", 'जङ्गलमा कुनै प्राणीले औषधीय वनस्पति प्रयोग गरेर चोटपटकको उपचार गरेको यो पहिलो रेकर्ड हो। ', ' ', 'टिकटकले अमेरिकाले प्रतिबन्ध पुष्टि गर्न “अनुमानका आधारमा चिन्ता” व्यक्त गरेको आरोप लगाउँदै उक्त कदम रोक्न अदालतसँग माग गरेको छ।', "पुरातत्त्वसम्बन्धी विशेषज्ञ चित्रकारहरूले सयौँ टुक्राहरू जोडेर बनाइएको खप्परका आधारमा 'नीयान्डर्टाल' महिलाको थ्री-डी मोडल बनाएका छन्। ", 'प्री-मनसुनको समयमा नेपालमा चट्याङ र हावाहुरीका घटनाहरू धेरै हुने गरेको विज्ञहरू बताउँछन्।', 'त्रिसट्ठी वर्षीया चर्चित कलाकारले क्यान्सरविरुद्ध आफ्नो सङ्घर्ष र रङ्गमञ्च एवं सामाजिक सञ्जालमार्फत् आफ्नो यात्राबारे जनचेतना जगाउने निर्णयबारे बीबीसीसँग कुराकानी गरेकी छन्।', 'आस्ट्राजेनेका खोपले कोभिड महामारीका क्रममा लाखौं मनिसहरूको ज्यान बचाएको विश्वास गरिन्छ, तर सँगसँगै यसले केही दुर्लभ रक्तजन्य घातक समस्या पनि निम्त्याएको बताइन्छ। ', 'गत वर्ष अर्थात् सन् २०२३ मा २० हजार जनाभन्दा बढी नेपाली कोरिया गएका थिए। कोरियामा अहिले नेपाली कामदारको सङ्ख्या ५५ हजार जनाभन्दा धेरै रहेको त्यहाँस्थित नेपाली दूतावासले जनाएको छ।', 'एमालेका हिक्मत कार्की मुख्यमन्त्री नियुक्त भएको दिन नै त्यसविरुद्ध मुद्दा लिएर कांग्रेसका केदार कार्की सर्वोच्च अदालत पुगेका छन्।', "महत्त्वाकाङ्क्षी 'नीअम' परियोजनाअन्तर्गत 'द लाइन' मरुभूमि सहरका लागि जग्गा खाली गर्ने काममा घातक बल प्रयोग गर्न साउदी अरबका अधिकारीहरूले अनुमति दिएको एक भूतपूर्व गुप्तचर अधिकारीले दाबी गरेका छन्।\n", 'कडा कार्य संस्कृति प्रोत्साहन गर्ने टिप्पणीलाई लिएर चिनियाँ प्रविधि कम्पनीकी जनसम्पर्क अधिकारीले किन माफी माग्नु पर्\u200dयो।', "केन्द्रमा गत फागुनमा सत्ता समीकरण परिवर्तन भएसँगै त्यसको प्रभाव प्रदेशहरूमा परिरहेको देखिएका बेला जसपा विभाजनले विशेषगरी मधेश प्रदेश सरकार 'ढल्न सक्ने' अनुमानहरू पनि गरिएका छन्।", 'अदालतले उनलाई चुनाव प्रचारप्रसार गर्न कुनै खालको रोक नलगाएको र आदेशमा उनले के गर्न पाउने वा नपाउने भन्ने कुनै विषयबारे उल्लेख नगरिएको पनि केजरीवालका वकिलले बताए।', 'इजरेली प्रधानमन्त्रीले गाजाको रफाहमा पूर्ण स्तरको आक्रमणलाई अनुमति दिएको खण्डमा हतियारको आपूर्ति रोक्न सकिने अमेरिकी चेतावनीलगत्तै बेन्जमिन नेतन्याहुले इजरेल "एक्लै खडा हुन सक्ने" प्रतिक्रिया दिएका छन्।', 'ब्रजिलको रिओ ग्रान्डी डु सुल प्रान्तमा आएको बाढी र पहिरोका कारण धनजनको ठूलो क्षति भएको छ।', 'खासगरी प्रधानमन्त्री, मुख्यमन्त्री एवं प्रदेश प्रमुख परिवर्तन भइरहने अनि मनपरी ढङ्गले आफू अनुकूलका व्यक्तिका तस्बिर राख्दा त्यसको आर्थिकभार राज्यकोषमा पर्ने गरेको जानकारहरू बताउँछन्।', 'नेपाली कांग्रेसले गृहमन्त्री रवि लामिछाने सहकारी ठगीमा संलग्न भएको विवरण सञ्चारमाध्यममा आइरहेको भन्दै उनीमाथि पनि छानबिन हुने गरी संसदीय समिति गठन गरिनुपर्ने अडानमा छ। ', 'हजारौँ आप्रवासीहरूलाई तस्करी गरी यूके पुर्\u200dयाएका स्करपीअन उपनामले चर्चित बार्जान बीबीसीको एउटा अनुसन्धानपछि पक्राउ गरेका छन्। ', 'अमेरिकाका पूर्वराष्ट्रपति डोनल्ड ट्रम्पविरुद्ध परेको मुद्दामा उनका पूर्ववकिल माइकल कोएनले ट्रम्पविरुद्ध बयान दिएका छन्। उनले के भने र उनको बयान किन महत्त्वपूर्ण छ?', 'नेपालमा जेठ १५ गते बजेट ल्याउने नियम छ। त्यसअघि संसद्\u200cमा नीति तथा कार्यक्रम प्रस्तुत हुन्छ। यसपालि मङ्गलवार संसद्को दुवै सदनको संयुक्त बैठक बस्दैछ र त्यसमा राष्ट्रपति रामचन्द्र पौडेलले नीति तथा कार्यक्रम प्रस्तुत गर्ने कार्यक्रम तय भएको छ।', 'उत्तर कोरियाको एउटा प्रान्तमा मानिसहरू भोकभोकै मर्न थालेको सुनेपछि देश छाडेर दक्षिण कोरियामा शरण लिन पुगेका यी व्यक्तिले सन् २०१५ देखि बोतलमा अन्य चिजसँगै चामल भरेर पठाउन थालेका हुन्।\n', "पिनाइल क्यान्सर भनिने पुरुषहरूमा लिङ्गको क्यान्सरका एकजना बिरामी जोआओ भन्छन्, रोगको निदान भएपछि लिङ्ग काटेर हटाउनुपर्ने अवस्थाबाट उनी 'निकै आत्तिएका' थिए।", 'सर्वोच्च अदालतको संवैधानिक इजलासको फैसलाको पूर्णपाठमा हदबन्दीभन्दा बढी भएको जग्गा बाँझो राखिएको भेटिए सरकारका नाममा ल्याउन समेत भनिएको छ।', 'इन्टरनेटमा बालबालिकाले कलिलै उमेरमा पहुँच पाउँदा उनीहरू गम्भीर जोखिमको नजिक पुगेको विशेषज्ञहरूले बताएका छन्। ती जोखिमको रोकथामका लागि तपाईँहामी के गर्न सक्छौँ?', 'मृत्यु सन्निकट हुँदा मानिसहरूले गर्ने अनुभूतिबारे डा क्रिस्टोफर केरले अध्ययन गरेका छन्। त्यस्ता मानिसहरूले अन्तिम समयमा अनुभव गर्ने दृश्य र तिनको अर्थबारे उनले कुरा गरेका छन्।', "आत्मसम्मान कम भएका व्यक्तिमा  'कम्प्लिमेन्ट' अर्थात प्रशंसाले चिन्ता बढाउन सक्छ किनभने उनीहरूको स्वधारणामा त्यसरी भएको तारिफले चुनौती दिन सक्छ। ", 'सन् २०२२ मा जोन म्याकफललाई शारीरिक अपाङ्गता भएका प्रथम अन्तरिक्षयात्रीको उम्मेदवारका रूपमा चयन गरिएको थियो। ', "थाईल्यान्डमा 'लीज म्याजस्टी' कानुनले राजतन्त्रको आलोचना गर्न प्रतिबन्ध लगाएको छ। राजतन्त्रको संरक्षणका लागि बनाइएको त्यस्तो कानुनलाई विश्वकै कठोर कानुनमध्ये मानिन्छ।", "हजारौँ आप्रवासीहरूलाई तस्करी गरी यूके पुर्\u200dयाएका 'स्कोर्पिअन' उपनामले चर्चित बर्जान मजिद बीबीसीको एउटा अनुसन्धानपछि इराकमा पक्राउ परेका छन्। ", 'नयाँ नोट छपाइमा जानुअघि अन्तर्राष्ट्रिय बोलकबोलसहितका कतिपय कानुनी प्रक्रियाहरू पूरा गर्नुपर्ने अधिकारीहरू बताउँछन्। ', 'अमेरिका, चीन र भारतले चन्द्रमामा मानव पठाउने योजना सार्वजनिक गरेका छन्। अन्तरिक्ष अभियानमा यी देशले किन अर्बौँ डलर लगानी गरेका हुन् र उनीहरूको अपेक्षा के छ?', 'जबरजस्ती करणी मुद्दामा उच्च अदालत पाटनले सफाइ दिएको केही घण्टापछि नेपाल क्रिकेट सङ्घ क्यानको निर्णय सार्वजनिक भएको हो।', ' प्रधानमन्त्रीको टाउको र छातीमा गोली लागेको देखिएको थियो। प्रधानमन्त्रीका सुरक्षाकर्मीमध्ये तीन जनाले उनको उद्धार गरी कारभित्र लगेका थिए।', 'मसला उत्पादन र निर्यातमा भारतको वर्चस्व छ। तर विभिन्न देशमा भारतीय मसलाको गुणस्तरबारे प्रश्न उठेको छ।', 'हीरामन्डीमा प्रमुख भूमिका रहेको मल्लिका जानको अभिनय गरेर फिल्म क्षेत्रमा वाहवाही पाइरहेकी मनीषा कोइरालाले बलुवुडकी चर्चित कलाकार रेखाले आफ्नो प्रशंसा गरेको सुन्दा आँशु आएको बताएकी छन्। रेखाले हीरामन्डी हेरेपछि मनीषालाई के भनिन् र उनको आँखा रसाए?', "'लेडी बुशरा' नामले चिनिने अमिर डीन यूकेका सर्वाधिक चर्चित 'ड्र्याग कलाकार'मध्ये एक हुन्।", "साउदी अरबलाई आधुनिक बनाउने युवराज मोहम्मद बिन सलमानको चाहनाअनुसार 'नीअम' परियोजना बनाउन लागिएको हो। \n\n", 'कृषि जनशक्तिको चरम अभावमाझ राष्ट्रपतिले घोषणा गरेको महत्त्वाकाङ्क्षी कृषिमा लगानी दशक कार्यान्वयनमा नआउँदै गम्भीर प्रश्नहरू तेर्सिएका छन्।', 'युक्रेनमा जारी युद्धमा रुसलाई सघाएको आरोप चीनलाई लागेकै बेला भ्लादिमिर पुटिनले फेरि बेइजिङ भ्रमणमा पुगेका छन्।', 'भारतमा निर्वाचनका बेला डीपफेक र एआईबाट सिर्जित भ्रामक सामग्रीको बिगबिगी बढेपछि विज्ञहरूले त्यसबाट पर्ने प्रभावबारे चिन्ता व्यक्त गरेका छन्।', 'जबरजस्ती करणी मुद्दामा उच्च अदालत पाटनले सफाइ दिएको केही घण्टापछि नेपाल क्रिकेट सङ्घ (क्यान) को निर्णय सार्वजनिक भएको हो।', 'उत्तर अमेरिकामा यो पहिलो विश्वकप हुँदै छ भने आयोजकको हैसियतले अमेरिकाले पनि सीधै विश्वकप खेल्ने मौका पाएको छ।', 'सहकारी संस्थाका सञ्चालकहरूले सम्पत्ति ब्याङ्कमा धितो राखेर ऋण लिएको पाइएका कारण लिलामी प्रक्रियामा समस्या भएको अधिकारीहरूले बताएका छन्।', "'ब्रेकिङ'लाई ओलिम्पिक्समा पहिलो पटक समावेश गरिएको हो। यसपालि नै तालिबानको धम्कीका बीच 'ब्रेकिङ' सिकेकी मनिजा तलाश शरणार्थी टोलीकी सदस्यका रूपमा खेल्दै छिन्।"]

#     # Preprocess the data
#     cleaned_data = [preprocess_text(doc) for doc in data]
#     print(cleaned_data)

#     # Further steps, like tokenization or saving the cleaned data, can follow


# Train the CharBPETokenizer

In [ ]:
!ls

'0. download_data.ipynb'     clean_nepberta_data.zip		 NepaliBPE
'1. train_tokenizer.ipynb'   IRIISNEPAL_Nepali_Text_Corpus.csv	 nepberta_text.txt
 clean_date_categories.csv   iriis_text.txt


## 1. Save data to text file

### 0. Get text from these unique urls (iriisnepal)

In [ ]:
import pandas as pd
import time

start_time = time.time()
# Input CSV file and output text file paths
input_csv = "IRIISNEPAL_Nepali_Text_Corpus.csv"
output_txt = "iriis_text.txt"

rows_count = 0
# Open the output file for writing
with open(output_txt, "w", encoding="utf-8") as output_file:
    # Process the CSV file in chunks
    chunk_size = 50000 if rows_count<5500000 else 10000  # Adjust the chunk size based on your memory capacity
    for chunk in pd.read_csv(input_csv, chunksize=chunk_size, usecols=["Article"]):

        # Apply the cleaning function to the "Article" column
        chunk["Article"] = chunk["Article"].apply(data_cleaner.clean_data)

        # Append the endoftext token
        chunk["Article"] = chunk["Article"] + " <|endoftext|>"

        # Write each chunk to the text file
        chunk["Article"].to_csv(output_file, index=False, header=False)

        rows_count += chunk_size
        if rows_count % 500000 == 0 or rows_count == 50000:
          print(rows_count, end='\n\n')
print(f'done! total rows: {rows_count}, \n time:{(time.time()-start_time)/3600} hours.')

<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


50000



<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks mo

500000



<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks mo

...

5500000



<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks mo

6000000



<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-7-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks mo

done! total rows: 6400000, 
 time:6.010486876567205 hours.


### 1. Get unique urls if nepberta- that are not in iriisnepal dataset

In [ ]:
import pandas as pd
import time
from urllib.parse import urlparse

unique_links=set()

# Input CSV file and output text file paths
input_csv = "clean_date_categories.csv"
output_txt = "nepberta_text.txt"

rows_count=0
saved_rows_count = 0
# Open the output file for writing
with open(output_txt, "w", encoding="utf-8") as output_file:
    # Process the CSV file in chunks
    chunk_size = 55000  # Adjust the chunk size based on your memory capacity
    for chunk in pd.read_csv(input_csv, chunksize=chunk_size, usecols=["link"]):
        # Filter rows where is_not_in_iriis returns True
        chunk["link"].apply(unique_links.add)
len(unique_links)

2760920

In [ ]:
unique_netloc=set()
error_links = []
for url in unique_links:
  try:
    unique_netloc.add(urlparse(url).netloc.replace('www.',''))
  except Exception as Ex:
      print(Ex)
      error_links.append(url)

print(f'error:{len(error_links)} \n unique_nepberta_netloc: \n\n {unique_netloc}')

error:0 
 unique_nepberta_netloc: 

 {'ekantipur.com', 'thahakhabar.com', 'dhangadhikhabar.com', 'onlinekhabar.com', 'dcnepal.com', 'pahilopost.com', 'shilapatra.com', 'nayapage.com', 'baahrakhari.com', 'nayapatrikadaily.com', 'dainiknewsnepal.com', 'hamrakura.com', 'emountaintv.com', 'kalakarmi.com', 'gorkhapatraonline.com', 'ratopati.com', 'hamrokhelkud.com', 'nepalsamaya.com', 'kathmandupress.com', 'reportersnepal.com', 'kendrabindu.com', 'setopati.com', 'imagekhabar.com', 'lokaantar.com', 'lokpath.com', 'dainikonline.com', 'ujyaaloonline.com', 'nepalpress.com', 'farakdhar.com', 'nepallive.com', 'nepalkhabar.com', 'himalkhabar.com', 'abhiyandaily.com', 'khabarhub.com', 'dev.setoparty.com', 'everestdainik.com', 'nagariknews.nagariknetwork.com'}


In [ ]:
iriis_dataset_urls=['khabarhub.com', 'setokhari.com', 'arghakhanchi.com', 'janaaastha.com', 'nepalijanta.com', 'safalnews.com', 'lokpath.com', 'avenues.tv', 'samudrapari.com', 'arthasarokar.com', 'ujyaaloonline.com', 'kendrabindu.com', 'nepalviews.com', 'purbelinews.com', 'imagekhabar.com', 'pardafas.com', 'daunnenews.com', 'souryaonline.com', 'sansarnews.com', 'sancharkendra.com', 'nayanepalnews.com', 'hamrakura.com', 'mysansar.com', 'rajdhanidaily.com', 'reportersnepal.com', 'canadanepal.com', 'chitawan.com', 'hamrokhelkud.com', 'nepalkhabar.com', 'aajakonews.com', 'bizshala.com', 'shikharnews.com', 'kharibot.com', 'enepalese.com', 'emountaintv.com', 'makalukhabar.com', 'brtnepal.com', 'nepalghatana.com', 'nepalpress.com', 'kakhara.com', 'kathmandupati.com', 'annapurnapost.com', 'diyopost.com', 'capitalnepal.com', 'technologykhabar.com', 'healthpati.com', 'samacharpati.com', 'ekantipur.com', 'pokharanews.com', 'spacesamachar.com', 'onlinekhabar.com', 'hellokhabar.com', 'saralpatrika.com', 'khojtalashonline.com', 'onlinetvnepal.com', 'barnanmedia.com', 'hamrobiratnagar.com', 'mahendranagarpost.com', 'ratopati.com', 'nepalgunjnews.com', 'madheshvani.com', 'nepallive.com', 'nonstopkhabar.com', 'techpana.com', 'nagariknews.com', 'nagariknews.nagariknetwork.com', 'ejanakpuronline.com', 'bhaktapurpost.com', 'eadarsha.com', 'shilapatra.com', 'nepalsamaya.com', 'ictsamachar.com', 'sagarmatha.tv', 'narimag.com.np', 'abcnepal.tv', 'sunaulonepal.com', 'hamrokhotang.com', 'bisalnepal.com', 'nepalihealth.com', 'nepalwatch.com', 'baahrakhari.com', 'kantipath.com', 'etajakhabar.com', 'nepalipatra.com', 'newsofnepal.com', 'himalkhabar.com', 'dcnepal.com', 'gorkhapatraonline.com', 'everestdainik.com', 'setopati.com', 'kathmandupress.com', 'farakdhar.com', 'karobardaily.com', 'palikakhabar.com', 'onlinemajdoor.com', 'dainiknepal.com', 'newskarobar.com', 'pahilopost.com', 'pariwartankhabar.com', 'drishtinews.com']
unique_nepberta_urls = [url for url in unique_netloc if url not in iriis_dataset_urls]
unique_nepberta_urls

['thahakhabar.com',
 'dhangadhikhabar.com',
 'nayapage.com',
 'nayapatrikadaily.com',
 'dainiknewsnepal.com',
 'kalakarmi.com',
 'lokaantar.com',
 'dainikonline.com',
 'abhiyandaily.com',
 'dev.setoparty.com']

## 2. Get text from these unique urls (nepberta-iriisnepal)

In [ ]:
import pandas as pd
import time
from urllib.parse import urlparse

start_time=time.time()
saved_netloc = set()
# filter urls from nepberta that are present in iriisnepal datasest
iriis_dataset_urls = ['ekantipur.com',  'onlinekhabar.com',  'nagariknews.com',  'thahakhabar.com',  'ratopati.com',  'reportersnepal.com',  'setopati.com',  'hamrakura.com',  'lokpath.com',  'abhiyandaily.com',  'pahilopost.com',  'lokaantar.com',  'dcnepal.com',  'nayapage.com',  'nayapatrikadaily.com',  'everestdainik.com',  'imagekhabar.com',  'shilapatra.com',  'khabarhub.com',  'baahrakhari.com',  'ujyaaloonline.com',  'nepalkhabar.com',  'emountaintv.com',  'kathmandupress.com',  'farakdhar.com',  'kendrabindu.com',  'dhangadhikhabar.com',  'gorkhapatraonline.com',  'nepalpress.com',  'hamrokhelkud.com',  'himalkhabar.com',  'nepallive.com',  'nepalsamaya.com',  'kalakarmi.com',  'dainiknewsnepal.com']
def is_not_in_iriis(url):
    try:
        # Extract the netloc from the URL
        netloc = urlparse(url).netloc.replace('www.','')
        return netloc in unique_nepberta_urls
        # return netloc not in iriis_dataset_urls
    except Exception as ex:
        print(f'\n\n error:{url}{ex}\n\n')
        # Handle invalid or missing URLs by excluding them
        return False

# Input CSV file and output text file paths
input_csv = "clean_date_categories.csv"
output_txt = "nepberta_text.txt"

rows_count=0
saved_rows_count = 0
# Open the output file for writing
with open(output_txt, "w", encoding="utf-8") as output_file:
    # Process the CSV file in chunks
    chunk_size = 45000  # Adjust the chunk size based on your memory capacity
    for chunk in pd.read_csv(input_csv, chunksize=chunk_size, usecols=["text", "link"]):
        # Filter rows where is_not_in_iriis returns True
        filtered_chunk = chunk[chunk["link"].apply(is_not_in_iriis)].dropna()
        chunk["link"].apply(saved_netloc.add)
        # Apply the cleaning function
        filtered_chunk["text"] = filtered_chunk["text"].apply(data_cleaner.clean_data)
        # Append the endoftext token
        filtered_chunk["text"] = filtered_chunk["text"] + " <|endoftext|>"
        # Write the filtered and cleaned rows to the text file
        filtered_chunk["text"].to_csv(output_file, index=False, header=False)

        rows_count += chunk_size
        saved_rows_count += len(filtered_chunk)
        if rows_count % 500000 == 0 or rows_count <= 55000:
          print(f'{(time.time()-start_time)/60}min. total rows: {rows_count}, \n saved_rows:{saved_rows_count}', end='\n\n')
print(f'done! total rows: {rows_count}, \n saved_rows:{saved_rows_count} \n time taken:{(time.time()-start_time)/60} minutes.')
print(saved_netloc)

0.1456180493036906min. total rows: 55000, 
 saved_rows:0



<ipython-input-3-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-3-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-3-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-3-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
<ipython-input-3-c4b837329946>:24: MarkupResemblesLocatorWarning: The input looks mo

done! total rows: 2805000, 
 saved_rows:570086 
 time taken:45.1467861533165 minutes.
Buffered data was truncated after reaching the output size limit.

In [ ]:
!ls -l --block-size=M

total 71897M
-rw------- 1 root root     1M Dec 18 15:01 '0. download_data.ipynb'
-rw------- 1 root root     1M Dec 21 05:28 '1. train_tokenizer.ipynb'
-rw------- 1 root root 13508M Dec 18 12:23  clean_date_categories.csv
-rw------- 1 root root  2672M Dec 18 12:11  clean_nepberta_data.zip
-rw------- 1 root root 26449M Dec 18 14:40  IRIISNEPAL_Nepali_Text_Corpus.csv
-rw------- 1 root root 26240M Dec 20 16:38  iriis_text.txt
drwx------ 2 root root     1M Dec 21 03:40  NepaliBPE
-rw------- 1 root root  3030M Dec 21 03:41  nepberta_text.txt


In [ ]:
'''
# Paths to the CSV files
nepberta_csv = "clean_date_categories.csv"
iriisnepal_csv = "IRIISNEPAL_Nepali_Text_Corpus.csv"

# filter urls from nepberta that are present in iriisnepal datasest
iriis_dataset_urls = ['ekantipur.com',  'onlinekhabar.com',  'nagariknews.com',  'thahakhabar.com',  'ratopati.com',  'reportersnepal.com',  'setopati.com',  'hamrakura.com',  'lokpath.com',  'abhiyandaily.com',  'pahilopost.com',  'lokaantar.com',  'dcnepal.com',  'nayapage.com',  'nayapatrikadaily.com',  'everestdainik.com',  'imagekhabar.com',  'shilapatra.com',  'khabarhub.com',  'baahrakhari.com',  'ujyaaloonline.com',  'nepalkhabar.com',  'emountaintv.com',  'kathmandupress.com',  'farakdhar.com',  'kendrabindu.com',  'dhangadhikhabar.com',  'gorkhapatraonline.com',  'nepalpress.com',  'hamrokhelkud.com',  'himalkhabar.com',  'nepallive.com',  'nepalsamaya.com',  'kalakarmi.com',  'dainiknewsnepal.com']
def is_not_in_iriis(url):
    try:
        # Extract the netloc from the URL
        netloc = urlparse(url).netloc
        return netloc not in iriis_dataset_urls
    except Exception:
        # Handle invalid or missing URLs by excluding them
        return False

# Function to stream text from CSV files
def text_generator1():
    # Stream text from the iriisnepal_csv file
    with open(iriisnepal_csv, 'r', encoding='utf-8') as f2:
        reader = csv.DictReader(f2)
        for row in reader:
            # print('21')
            if 'Article' in row and row['Article']:
                # print('22')
                yield data_cleaner.clean_data(row['Article'])
                # print(data_cleaner.clean_data(row['Article']))
            # else:print(row)
            # break
def text_generator2():
    # Stream text from the nepberta_csv file
    with open(nepberta_csv, 'r', encoding='utf-8') as f1:
        reader = csv.DictReader(f1)
        for row in reader:
            if 'text' in row and row['text'] and 'link' in row and row['link']:
                # print('11')
                if is_not_in_iriis(row['link']):
                    # print('12')
                    # print(row['text'])
                    yield data_cleaner.clean_data(row['text'])
                # break

with open('iriis_text.txt', 'w', encoding='utf-8') as out_f:
    for text in text_generator1():
        out_f.write(text + '<|endoftext|>')  # Write each cleaned article to the file

with open('nepberta_text.txt', 'w', encoding='utf-8') as out_f:
    for text in text_generator2():
        out_f.write(text + '<|endoftext|>')  # Write each cleaned article to the file

print(f"Filtered text saved to 'iriis_text.txt', 'nepberta_text.txt' ")
'''

## 2. Train CharBPETokenizer
* **ByteLevelBPETokenizer:** use UTF-8 encoding as fundamental unit of tokenization
* **CharLevelTokenizer:** uses character as fundamental unit of tokenization.[link text](https://)

In [ ]:
!rm -rf NepaliBPE

In [ ]:
import csv
import os
import time
from tokenizers import CharBPETokenizer
from urllib.parse import urlparse

start_time = time.time()
# Output directory for the tokenizer
output_dir = "NepaliBPE"
os.makedirs('NepaliBPE', exist_ok=True)


# Initialize a BPE tokenizer
tokenizer = CharBPETokenizer()
tokenizer.model.unk_token="<|unk|>"

# Train the tokenizer on the saved text file
tokenizer.train(
    files=["iriis_text.txt", "nepberta_text.txt"],
    vocab_size=50_000,
    min_frequency=10,
    special_tokens=["<|endoftext|>", "<|unk|>"]
)

# # Train the tokenizer on streamed text
# tokenizer.train_from_iterator(
#     text_generator(),
#     vocab_size=50_000,  # Adjust vocabulary size as needed
#     min_frequency=10,    # Ignore tokens that appear less frequently
#     special_tokens=["<|endoftext|>", "<|unk|>"]
# )

# Save the tokenizer to the output directory
tokenizer.save_model(output_dir)
print(f"Tokenizer saved to {output_dir}")
print(f"time_taken:{(time.time()-start_time)/60} minutes.")

Tokenizer saved to NepaliBPE
time_taken:255.33936193386714 minutes.


## load trained tokenizer


In [133]:
%cd /content/drive/MyDrive/Research/dataset/hf_dataset/NepaliBPE

/content/drive/MyDrive/Research/dataset/hf_dataset/NepaliBPE


### save `tokenizer.json`

In [170]:
from tokenizers import CharBPETokenizer

# Load the trained tokenizer
tokenizer = CharBPETokenizer(
    "vocab.json",
    "merges.txt"
)
# print(tokenizer.encode("नेपाल विद्युत प्राधिकरणका कार्यकारी निर्देशक कुलमान").tokens)
tokenizer.model.unk_token="<|unk|>"
special_tokens = [
            "<|begin_of_text|>",
            "<|end_of_text|>",
            "<|start_header_id|>",
            "<|end_header_id|>",
            "<|eot_id|>", # tf is this
            "\n\n",
            # "<|unk|>" # it is already present in vocab
        ]
tokenizer.add_special_tokens(special_tokens)

print(tokenizer.encode('fuck').tokens)
tokenizer.save('tokenizer.json')

['f', '<|unk|>', '<|unk|>', '<|unk|>']


In [172]:
tokenizer.encode("भूमिका प्रयोगकर्ता सन्दर्भ").tokens

['भूमिका</w>', 'प्रयोगकर्ता</w>', 'सन्दर्भ</w>']

### load using `PrePrainedTokenizerFast`

In [152]:
# load using PreTrainedTokenizerFast
from transformers import PreTrainedTokenizerFast
# # Save as tokenizer.json
# tokenizer.save("NepaliBPE/tokenizer.json")
# Load the tokenizer
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json")

# Test the tokenizer
text = "गत आइतबार साँझपख केही आत्मीय मित्रहरूसँग भलाकुसारीमा थिएँ । गफैगफमा राजनीति र खेलकुदका प्रसंग उठे । त्यत्तिकैमा नेपालमा हुने गरेका साहित्य महोत्सव र वक्ताहरूका बारेमा एकछिन भुलियो । जयपुर साहित्य महोत्सवको कुरा पनि निस्कियो ।"  # Example Nepali text
tokens = fast_tokenizer.tokenize(text)
print(tokens)
fast_tokenizer.tokenize('<|fuck|>')


['गत</w>', 'आइतबार</w>', 'साँझपख</w>', 'केही</w>', 'आत्मीय</w>', 'मित्र', 'हरूसँग</w>', 'भलाकु', 'सा', 'रीमा</w>', 'थिएँ</w>', '।</w>', 'ग', 'फै', 'ग', 'फमा</w>', 'राजनीति</w>', 'र</w>', 'खेलकुदका</w>', 'प्रसंग</w>', 'उठे</w>', '।</w>', 'त्यत्ति', 'कैमा</w>', 'नेपालमा</w>', 'हुने</w>', 'गरेका</w>', 'साहित्य</w>', 'महोत्सव</w>', 'र</w>', 'वक्ता', 'हरूका</w>', 'बारेमा</w>', 'एकछिन</w>', 'भु', 'लियो</w>', '।</w>', 'जय', 'पुर</w>', 'साहित्य</w>', 'महोत्सवको</w>', 'कुरा</w>', 'पनि</w>', 'निस्कियो</w>', '।</w>']


['<</w>', '|</w>', 'f', '<|unk|>', '<|unk|>', '<|unk|>', '|</w>', '></w>']

In [159]:
tokens = fast_tokenizer.encode('<|begin_of_text|>') # [50000]
tokens += fast_tokenizer.encode("गत आइतबार साँझपख")
tokens

[50000, 576, 1600, 28392]

In [2]:
from transformers import PreTrainedTokenizerFast

text = "गत आइतबार साँझपख केही आत्मीय मित्रहरूसँग भलाकुसारीमा थिएँ । गफैगफमा राजनीति र खेलकुदका प्रसंग उठे । त्यत्तिकैमा नेपालमा हुने गरेका साहित्य महोत्सव र वक्ताहरूका बारेमा एकछिन भुलियो । जयपुर साहित्य महोत्सवको कुरा पनि निस्कियो ।"  # Example Nepali text
tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")
tokenizer.tokenize(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.53M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

['गत</w>',
 'आइतबार</w>',
 'साँझपख</w>',
 'केही</w>',
 'आत्मीय</w>',
 'मित्र',
 'हरूसँग</w>',
 'भलाकु',
 'सा',
 'रीमा</w>',
 'थिएँ</w>',
 '।</w>',
 'ग',
 'फै',
 'ग',
 'फमा</w>',
 'राजनीति</w>',
 'र</w>',
 'खेलकुदका</w>',
 'प्रसंग</w>',
 'उठे</w>',
 '।</w>',
 'त्यत्ति',
 'कैमा</w>',
 'नेपालमा</w>',
 'हुने</w>',
 'गरेका</w>',
 'साहित्य</w>',
 'महोत्सव</w>',
 'र</w>',
 'वक्ता',
 'हरूका</w>',
 'बारेमा</w>',
 'एकछिन</w>',
 'भु',
 'लियो</w>',
 '।</w>',
 'जय',
 'पुर</w>',
 'साहित्य</w>',
 'महोत्सवको</w>',
 'कुरा</w>',
 'पनि</w>',
 'निस्कियो</w>',
 '।</w>']

* very similar to our previous tokenizer

# LLAMA3 format compatible: Put it inside a Tokenizer class

In [193]:
%cd /content/drive/MyDrive/Research/dataset/hf_dataset/NepaliBPE
!ls

/content/drive/MyDrive/Research/dataset/hf_dataset/NepaliBPE
merges.txt  tokenizer.json  vocab.json


In [194]:
import os

from transformers import PreTrainedTokenizerFast
class Tokenizer:
    def __init__(self, tokenizer_model_path):
        assert os.path.isfile(tokenizer_model_path), f"Tokenizer Model file {tokenizer_model_path} not found"

        # load the tokenizehere
        self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_model_path)

        # previously added
        # self.special_tokens = {
        #     "<|begin_of_text|>": 128000,
        #     "<|end_of_text|>": 128001,
        #     "<|start_header_id|>": 128006,
        #     "<|end_header_id|>": 128007,
        #     "<|eot_id|>": 128009,
        # }



    def encode(self, text, bos=False, eos=False):
        '''
        parameter: allowed_special removed
        parameter: disallowed_special removed
        '''
        if bos:
            # tokens = [self.special_tokens["<|begin_of_text|>"]]
            tokens = self.tokenizer.encode('<|begin_of_text|>') # [50000]
        else:
            tokens = []

        tokens += self.tokenizer.encode(text)

        if eos:
            # tokens.append(self.special_tokens["<|end_of_text|>"])
            tokens.append(self.tokenizer.encode('<|end_of_text|>')[0]) # [50001]
        return tokens

    def decode(self, tokens):
        # return self.model.decode(tokens)
        return self.tokenizer.decode(tokens)


class ChatFormat:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encode_header(self, message):
        tokens = []
        tokens.append(self.tokenizer.tokenizer.encode('<|start_header_id|>')[0]) # 50002
        tokens.extend(self.tokenizer.encode(message["भूमिका"], bos=False, eos=False))
        tokens.append(self.tokenizer.tokenizer.encode('<|end_header_id|>')[0])
        tokens.extend(self.tokenizer.encode("\n\n", bos=False, eos=False))
        # tokens.append(self.tokenizer.special_tokens["<|start_header_id|>"])
        # tokens.extend(self.tokenizer.encode(message["role"], bos=False, eos=False))
        # tokens.append(self.tokenizer.special_tokens["<|end_header_id|>"])
        # tokens.extend(self.tokenizer.encode("\n\n", bos=False, eos=False))
        return tokens

    def encode(self, text):
        message = {
            "भूमिका": "प्रयोगकर्ता",
            "सन्दर्भ": text
        }

        tokens = self.encode_header(message)
        tokens.extend(
            self.tokenizer.encode(message["सन्दर्भ"].strip(), bos=False, eos=False)
        )
        # tokens.append(self.tokenizer.special_tokens["<|eot_id|>"])
        tokens.append(self.tokenizer.tokenizer.encode('<|eot_id|>')[0])
        return tokens

    def decode(self, token_ids):
        return self.tokenizer.decode(token_ids)

tokenizer = Tokenizer("tokenizer.json")
chat_tokenizer = ChatFormat(tokenizer)

text = "नेपाल विद्युत प्राधिकरणका कार्यकारी निर्देशक कुलमान घिसिङले माथिल्लो अरुण जलविद्युत आयोजना विश्व बैंक र एडीबीबाट वित्तीय व्यवस्थापन नभए नेपाली जनताको लगानीमा बनाउने तयारी रहेको बताएका छन् ।"
# normal tokenizer
print([tokenizer.tokenizer.decode([token]) for token in tokenizer.encode(text)])

# formatted tokenizer
print([tokenizer.tokenizer.decode([token]) for token in chat_tokenizer.encode(text)])

['नेपाल', 'विद्युत', 'प्राधिकरणका', 'कार्यकारी', 'निर्देशक', 'कुलमान', 'घिसिङले', 'माथिल्लो', 'अरुण', 'जलविद्युत', 'आयोजना', 'विश्व', 'बैंक', 'र', 'एडी', 'बीबाट', 'वित्तीय', 'व्यवस्थापन', 'नभए', 'नेपाली', 'जनताको', 'लगानीमा', 'बनाउने', 'तयारी', 'रहेको', 'बताएका', 'छन्', '।']
['<|start_header_id|>', 'प्रयोगकर्ता', '<|end_header_id|>', '\n\n', 'नेपाल', 'विद्युत', 'प्राधिकरणका', 'कार्यकारी', 'निर्देशक', 'कुलमान', 'घिसिङले', 'माथिल्लो', 'अरुण', 'जलविद्युत', 'आयोजना', 'विश्व', 'बैंक', 'र', 'एडी', 'बीबाट', 'वित्तीय', 'व्यवस्थापन', 'नभए', 'नेपाली', 'जनताको', 'लगानीमा', 'बनाउने', 'तयारी', 'रहेको', 'बताएका', 'छन्', '।', '<|eot_id|>']


In [192]:
len(tokenizer.tokenizer)

50006

In [ ]:
chat_tokenizer.encode("लामा हरु ले के खान्छन् ?")